In [6]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [7]:
# Import the datasets, 
# Specify the delimeter, header and columns

#  Describes static characteristics of an account
df_account = pd.read_csv('account.csv', delimiter=';', header=0, names=['account_id', 'district_id', 'frequency', 'date'])

# Describes a credit card issued to an account
df_card = pd.read_csv('card.csv', delimiter=';', header=0, names=['card_id', 'disp_id', 'type', 'issued'])

# Describes characteristics of a client
df_client = pd.read_csv('client.csv', delimiter=';', header=0, names=['client_id', 'birth_number', 'district_id'])

# Relates together a client with an account
df_disp = pd.read_csv('disp.csv', delimiter=';', header=0, names=['disp_id', 'client_id', 'account_id', 'type'])

# Describes demographic characteristics of a district
df_district = pd.read_csv('district.csv', delimiter=';', header=0, names=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16'])

# Describes a loan granted for a given account
df_loan = pd.read_csv('loan.csv', delimiter=';', header=0, names=['loan_id', 'account_id', 'date', 'amount', 'duration', 'payments', 'status'])

# Describes characteristics of a payment order
df_order = pd.read_csv('order.csv', delimiter=';', header=0, names=['order_id', 'account_id', 'bank_to', 'account_to', 'amount', 'k_symbol'])

# Describes one transaction on an account
df_trans = pd.read_csv('trans.csv', delimiter=';', header=0, names=['trans_id', 'account_id', 'date', 'type', 'operation', 'amount', 'balance', 'k_symbol', 'bank', 'account'])

C:\Users\carol\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_district.rename(columns={'A1': 'district_id'}, inplace=True)

In [9]:
# Merge df_account and df_order by account_id
df_merged = pd.merge(df_account, df_order, on='account_id', how='left')
df_merged.drop(columns=['district_id'], inplace=True)

# Merge df_merged and df_trans by account_id
df_merged = pd.merge(df_merged, df_trans, on='account_id', how='left')

# Merge df_merged and df_disp by account_id
df_merged = pd.merge(df_merged, df_disp, on='account_id', how='left')

# Merge df_merged and df_client by client_id
df_merged = pd.merge(df_merged, df_client, on='client_id', how='left')

# Merge df_merged and df_district by district_id
df_merged = pd.merge(df_merged, df_district, on='district_id', how='left')

In [10]:
df_merged

,account_id,frequency,date_x,order_id,bank_to,account_to,amount_x,k_symbol_x,trans_id,date_y,...,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,576,POPLATEK MESICNE,930101,30253.0,OP,71033382.0,3662.0,SIPO,171812,930101,...,0,1,1,100.0,10673,4.75,5.44,100,18782,18347
1,576,POPLATEK MESICNE,930101,30253.0,OP,71033382.0,3662.0,SIPO,171812,930101,...,0,1,1,100.0,10673,4.75,5.44,100,18782,18347
2,576,POPLATEK MESICNE,930101,30253.0,OP,71033382.0,3662.0,SIPO,171813,930111,...,0,1,1,100.0,10673,4.75,5.44,100,18782,18347
3,576,POPLATEK MESICNE,930101,30253.0,OP,71033382.0,3662.0,SIPO,171813,930111,...,0,1,1,100.0,10673,4.75,5.44,100,18782,18347
4,576,POPLATEK MESICNE,930101,30253.0,OP,71033382.0,3662.0,SIPO,3549613,930131,...,0,1,1,100.0,10673,4.75,5.44,100,18782,18347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2243453,3276,POPLATEK MESICNE,971229,34262.0,WX,88365083.0,1017.0,POJISTNE,961640,981208,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
2243454,3276,POPLATEK MESICNE,971229,34262.0,WX,88365083.0,1017.0,POJISTNE,961634,981213,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
2243455,3276,POPLATEK MESICNE,971229,34262.0,WX,88365083.0,1017.0,POJISTNE,961634,981213,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
2243456,3276,POPLATEK MESICNE,971229,34262.0,WX,88365083.0,1017.0,POJISTNE,3639005,981231,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
